# Distribution network graph simplification

Copyright: Alyona Ivanova SLAC National Laboratory Feb 2019

This code uses gridlabd model .json dump to extract the information for a distribution network graph and completes two tasks: <br>
1. Series simplification of components <br>
2. Enumeration of nodes based on their level on the network tree

The instructions for extracting a json file from a gridlab-d model see link __https://github.com/dchassin/gridlabd/wiki/json__

First, the relevant python libraries are imported :

In [1]:
import networkx as nx 
import matplotlib.pyplot as plt
import json
import operator
%matplotlib inline

Defining the class for loading the json file as :

In [2]:
class GridlabdModel :
    def __init__(self,jsonfile):
        fh = open(jsonfile)
        self.model = json.load(fh)
        assert(self.model["application"]=='gridlabd')
        assert(self.model["version"]>='4.0.0')

This function processes the json file and finds all the objects in the model and categorizes them accordingly. Returns all the nodes with two branches and the edges that correlate to those nodes.

In [3]:
def object_def() : 
    model = GridlabdModel('gridlabd.json')
    substation_node = []
    nodes = {} # does not include the substation node
    links_from = []
    links_to = []
    links = []
    edges = {}
    switch_pos = []
    class_set = set()
    count_links = {}
    two_branch_nodes = {}
    two_branch_nodes_attr = {}
    two_branch_nodes_list = []
    two_branch_links_list = []
    two_edge_list_temp = []
    two_edge_list = []
    two_edge_node = {}
    simple_links = []
    
    for obj,data in model.model["objects"].items() :
        class_set.add(data["class"]) #set of all class types in the model
        # defining the node and the swing bus 
        if data["class"]=="node" or data["class"]=="meter" :
            if data["bustype"] == "SWING" :
                substation_node = obj
                nodes[obj]=data
            nodes[obj]=data
        if data["class"]=="triplex_load":
            nodes[obj]=data
        if data["class"] == "capacitor" :
            nodes[obj]=data
            links.append((data["parent"],obj))
         #defining the links 
        if data["class"] == "overhead_line" or  data["class"] == "underground_line"  or data["class"] == "transformer" or data["class"] == "switch":
            # assuming that every object is a node 
            if data["class"] == "switch" :
                if data["status"] == "OPEN" : 
                    print("switch open")
                else :
                    nodes[obj]=data
                    links.append((data["from"],obj)) # to node
                    links.append((obj,data["to"]))# from node
            else :
                nodes[obj]=data
                links.append((data["from"],obj)) # to node
                links.append((obj,data["to"]))# from node

        # check for lateral or OPEN switches
        if data["class"]=="switch":
            switch_pos.append(data["status"])
        # summary of line configurations 
        
    print(nodes)
    print(links)
    links_from, links_to = zip(*links)
    
    # counting the number of branches that each node has
    # i is each node 
    for i in nodes : 
        # Getting each pair of links that has the node in it and takes the length of the list 
        # This then determines the number of branch at each node
        # count_links - dictionary that contains all the nodes with their corresponding branch count
        count_links[i] =  len([item for item in links if i in item])

    # Compiling a new dictionary of nodes that have two branches and gathering their from and to coordinates
    for link,value in count_links.items() : 
        if value==2 : 
            two_branch_nodes_list.append(link)
            link_from = [link_from for link_from in links if link == link_from[1]]
            element_from = link_from[0][0]
            link_to = [link_to for link_to in links if link == link_to[0]]
            element_to = link_to[0][1]
            two_branch_nodes[link] = [element_from, element_to]
            two_branch_nodes_attr[link] = nodes[link]
            two_branch_links_list.append([element_from,element_to])
            two_edge_list_temp.append((element_from,link))
            two_edge_list_temp.append((link,element_to))
    two_edge_list_temp = set(two_edge_list_temp)

    for link in two_edge_list_temp : 
        if link[0] in two_branch_nodes_list : 
            if link[1] in two_branch_nodes_list :
                two_edge_list.append(link)
    return two_branch_nodes_attr, two_edge_list, nodes, links, substation_node


The following function uses all the nodes with the corresponding links as an input and returns all the graphs that are connected seperately. 

In [4]:
def series_graphs(two_branch_nodes_attr, two_edge_list) :
    G = nx.Graph()

    for node,attr in two_branch_nodes_attr.items() : 
        G.add_node(node, data = attr)
    G.add_edges_from(two_edge_list)
    graphs = list(nx.connected_component_subgraphs(G))
    # number of subgraphs to be compressed 
    len_of_subgraphs = len(graphs)-1
    return graphs

Defines the position for plotting. The plotting is done according to the level of the object in the distribution system.

In [5]:
def hierarchy_pos(G, root, levels=None, width=1., height=1.):
    '''If there is a cycle that is reachable from root, then this will see infinite recursion.
       G: the graph
       root: the root node
       levels: a dictionary
               key: level number (starting from 0)
               value: number of nodes in this level
       width: horizontal space allocated for drawing
       height: vertical space allocated for drawing'''
    TOTAL = "total"
    CURRENT = "current"
    def make_levels(levels, node=root, currentLevel=0, parent=None):
        """Compute the number of nodes for each level
        """
        if not currentLevel in levels:
            levels[currentLevel] = {TOTAL : 0, CURRENT : 0}
        levels[currentLevel][TOTAL] += 1
        neighbors = G.neighbors(node)
        for neighbor in neighbors:
            if not neighbor == parent:
                levels =  make_levels(levels, neighbor, currentLevel + 1, node)
        return levels

    def make_pos(pos, node=root, currentLevel=0, parent=None, vert_loc=0):
        dx = 1/levels[currentLevel][TOTAL]
        left = dx/2
        pos[node] = ((left + dx*levels[currentLevel][CURRENT])*width, vert_loc)
        levels[currentLevel][CURRENT] += 1
        neighbors = G.neighbors(node)
        for neighbor in neighbors:
            if not neighbor == parent:
                pos = make_pos(pos, neighbor, currentLevel + 1, node, vert_loc-vert_gap)
        return pos
    if levels is None:
        levels = make_levels({})
    else:
        levels = {l:{TOTAL: levels[l], CURRENT:0} for l in levels}
    vert_gap = height / (max([l for l in levels])+1)
    return make_pos({})

This function sounds the levels for each node and returns a list of nodes and the corresponding levels.

In [6]:
# Editing this to record the number of levels in the attributes 
def count_levels(G_p, node_levels, node, currentLevel=0, parent=None):
        """Compute the number of nodes for each level"""
        # if new level...
        if not currentLevel in node_levels:
            node_levels[node]=currentLevel
        # Find the neighbors of current node
        neighbors = G_p.neighbors(node) # returns a list of nodes connected to 'node'
        for neighbor in neighbors:
            if not neighbor == parent:
                node_levels = count_levels(G_p, node_levels, neighbor, currentLevel + 1, node)
        return node_levels
   

Function for plotting the original expanded graph.

In [7]:
# NOT SURE THIS IS NECESSARY
def original_graph(nodes, links) :
    G_org = nx.Graph()
    G_org.add_edges_from(links)
    for node,attr in nodes.items() : 
        G_org.add_node(node, data = attr)
   # G_org=nx.convert_node_labels_to_integers(G_org, ordering = "default")
    return G_org

Function that performs the simplification of the system. 

In [8]:
def graph_simplification(nodes, links, graphs, root_node) : 
    simple_nodes = nodes.copy() # dictionary
    simple_links = links.copy() # list


    # Adding the edges into a simplified graph
    for graph in graphs : 
        temp_list = list(graph.edges())
    # deleting the links that are associated with compression
        for i in range(len(temp_list)) : 
            if temp_list[i] in simple_links : 
                simple_links.remove(temp_list[i])
            if (temp_list[i][1], temp_list[i][0]) in simple_links: 
                simple_links.remove((temp_list[i][1], temp_list[i][0]))
                links.remove((temp_list[i][1], temp_list[i][0]))
                links.append((temp_list[i][0], temp_list[i][1]))
    # Adding the nodes into a simplified graph
    for graph in graphs : 
        for i in range(len(list(graph.nodes))) :  
            del simple_nodes[list(graph.nodes)[i]]
        # Assigning the first node name to the aggregate sum of collapsed nodes 
            for link in simple_links :
                if list(graph.nodes)[i] in link[1] : 
                    aggr_node = list(graph.nodes)[i]
        simple_nodes[aggr_node] = graph.nodes(data=True)

    simple_links_final = simple_links.copy()
    for graph in graphs :
        temp_list = list(graph.edges())
        for i in range(len(list(graph.nodes))) :  
            for link in simple_links :
                    if list(graph.nodes)[i] in link[1] : 
                        aggr_node = list(graph.nodes)[i]
        for ind,link in enumerate(simple_links) :
            for temp in temp_list :
                if temp[1] in link[0] :
                    simple_links_final[ind]=(aggr_node,link[1])
                if temp[0] in link[0] : 
                    simple_links_final[ind]=(aggr_node,link[1])

    
    

    G_p, G_p_sort_new = relabel_graph(simple_nodes, simple_links_final, root_node)
    
    print("Any cycles? %s " % nx.cycle_basis(G_p))
    print("Directed? %s " % nx.is_directed(G_p))
    print("Is it a tree? %s " % nx.is_tree(G_p))
    
    return G_p, G_p_sort_new


Function responsible for reordering the graph and then renaming the graph names to integer value. Note, if just renaming the nodes that are already in the desired order - this funcition is not required and only the nx.convert_node_labels_to_integers(G) is required (as seen in __main__) 

In [9]:
def relabel_graph(simple_nodes, simple_links_final, root_node) : 
    G_p = nx.OrderedGraph()
    for node,attr in simple_nodes.items() : 
        G_p.add_node(node, data = attr)
    G_p.add_edges_from(simple_links_final)
    G_p_sort = nx.OrderedGraph() #graph with sorted nodes 
    G_p_sort_new = nx.OrderedGraph()
    nodes_with_levels = {}
    final_levels = count_levels(G_p, {},root_node)
    final_levels = dict(sorted(final_levels.items(), key=operator.itemgetter(1)))
    # Populate the nodes dictionary with level numbers 
    
    for n,n_val in final_levels.items() : 
        nodes_with_levels[n] = {"level" : final_levels[n],"node_attr": simple_nodes[n]}

    for node, attr in nodes_with_levels.items() :
        G_p_sort.add_node(node, data = attr)

    # Note, the edges have to be added after the nodes to preserve the order 
    G_p_sort.add_edges_from(simple_links_final) 
    G_p_sort_new=nx.convert_node_labels_to_integers(G_p_sort)
    
    return G_p, G_p_sort_new

This function uses the original graph and cuts it. Returns the lower portion of the graph including the input node specified.

In [10]:
def graph_cut(G, node) : 
    # input: full graph that's enumerated and the node of interest 
    # Find the two neighbors of node 
    neighbor_nodes = list(G.neighbors(node))
    target_node_level = G.node[node]['data']['level']
    for i in range(len(neighbor_nodes)) : 
        level = G.node[neighbor_nodes[i]]['data']['level']
        if level < target_node_level : 
            G.remove_edge(node, neighbor_nodes[i])

    # list of two graphs
    graphs = list(nx.connected_component_subgraphs(G))
    # Check the levels of the nodes in each graph 
    for graph in graphs : 
        #  Find the subgraph that contains the node specified 
        if node in graph : 
            G = graph
    return G

Main function for execution of the program. Includes examples as to how to specify nodes and cuts. 

In [11]:
if __name__ == '__main__' :
    two_branch_nodes, two_branch_edges, nodes, links, substation_node = object_def()
    graphs = series_graphs(two_branch_nodes, two_branch_edges)
    G_org = original_graph(nodes, links)
    #G_p, G_p_enum = graph_simplification(nodes, links, graphs, substation_node)
    
    
    
    #### GRAPH PLOTTING ########
    # Plot original
    pos = hierarchy_pos(G_org,substation_node)
    nx.draw(G_org, pos=pos, node_size=1,with_labels=True, font_size=1, width = 0.1)

    plt.savefig('original_graph.png', dpi=1500)
    plt.clf()
    
   # node_of_interest = 1
  #  G = nx.OrderedGraph()
    #G = graph_cut(G_p_enum, node_of_interest)
    
    # FUNCTION FOR RE-ENUMERATING THE CUT GRAPH
    #G_enum=nx.convert_node_labels_to_integers(G)
   # G_enum = G
    
    # PLOT cut GRAPH
  #  pos_p = hierarchy_pos(G_enum,0)
   # nx.draw(G_enum, pos=pos_p, node_size=1,with_labels=True, font_size=1, width = 0.1)

 #   plt.savefig('cut_graph.png', dpi=1500)
 #   plt.clf()
    
#     #PLOT GRAPH WITH ORIGINAL OBJECT NAMES
#     pos_p = hierarchy_pos(G_p_sort,root_node)
#     nx.draw(G_p_sort, pos=pos_p, node_size=1,with_labels=True, font_size=1, width =0.01)
#     plt.savefig('heirarchy_p.png', dpi=1500) 
#     plt.clf()
    
#     pos_p = hierarchy_pos(G_enum,0)
#     nx.draw(G_enum, pos=pos_p, node_size=1,with_labels=True, font_size=1, width =0.1)
#     plt.savefig('graph_cut.png', dpi=1500) 
#     plt.clf()


switch open
switch open
{'line_node101_node102': {'id': '5', 'class': 'overhead_line', 'rank': '1', 'clock': '2018-01-30 00:00:00 EST', 'schedule_skew': '0', 'rng_state': '470211272', 'heartbeat': '0', 'guid': '0x2b7ad57a4a2827ad751659381bec8edd', 'flags': '0x100', 'configuration': 'lc_default', 'length': '+175 ft', 'status': 'CLOSED', 'from': 'node_101', 'to': 'node_102', 'power_in': '+103046+9.95923d VA', 'power_out': '+103038+9.9553d VA', 'power_out_real': '+101487 W', 'power_losses': '+10.3896+52.7736d VA', 'power_in_A': '+34970.2+12.5486d VA', 'power_in_B': '+32188.4+4.64119d VA', 'power_in_C': '+36088.6+12.188d VA', 'power_out_A': '+34966.9+12.5432d VA', 'power_out_B': '+32187.3+4.63835d VA', 'power_out_C': '+36085.3+12.1848d VA', 'power_losses_A': '+4.63822+57.337d VA', 'power_losses_B': '+1.96671+58.6798d VA', 'power_losses_C': '+3.85245+44.244d VA', 'current_out_A': '+10.7142-9.8942j A', 'current_out_B': '-12.1205-5.72671j A', 'current_out_C': '+3.20292+14.6904j A', 'current_i

<Figure size 432x288 with 0 Axes>